## Instance Struture

In [2]:
mutable struct Instance
    n :: Int64
    D
    l
    avg :: Float64
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        avg = 0
        if s != "STDIN"
            open(s) do f
                line = 1
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                nNodes = parse(Int64, lineSplited[1])
                l = parse(Int64, lineSplited[2])
                D = zeros(nNodes,nNodes)
                avgRatio = 1/((nNodes^2)-nNodes)

                while ! eof(f)
                    lineContent = readline(f)
                    lineSplited = split(lineContent, " ")
                    for col = 1:nNodes
                        value = parse(Float64, lineSplited[col])
                        D[line,col] = value
                        avg += value*avgRatio
                    end

                    line += 1
                end
            end
        else
            line = 1
            lineContent = readline("STDIN")
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            avgRatio = 1/((nNodes^2)-nNodes)

            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    value = parse(Float64, lineSplited[col])
                    D[line,col] = value
                    avg += value*avgRatio
                end

                line += 1
            end
        end
        new(nNodes,D,l,avg)
    end
end

## Solution Structure


In [3]:
mutable struct Solution
    π :: Array{Int64,1} ## solução: uma permutação de cidades
    v :: Float64        ## valor da solução: distância total
    Solution(s::Solution) = new(copy(s.π),s.v)
    Solution(π,v) = new(copy(π),v)
end
Base.copy(s::Solution) = Solution(s)

 ## Nearest Neighbor

In [4]:
function nearestNeighbor(I::Instance)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for k=2:I.n
      f=findall(free)
      j=f[findmin(I.D[s.π[end],f])[2]]
    #       s.v += I.D[s.π[end],j]
      push!(s.π,j)
      free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
#     push!(s.π,1)
    s
end

nearestNeighbor (generic function with 1 method)

## Random Nearest Neighbor

In [5]:
function randomNearestNeighbor(I::Instance,k)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for i=2:I.n
        f=findall(free)
        ncand=min(k,I.n-i+1)
        j=f[sortperm(I.D[s.π[end],f])[rand(1:ncand)]]
    #          s.v += I.D[s.π[end],j]
        push!(s.π,j)
        free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
#             println("$(s.π[i]) - $(s.π[((i+lookahead-1)%(I.n))+1])")
#             println("$(I.D[i,((i+lookahead-1)%(I.n))+1])")
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
#     push!(s.π,1)
    s
end

randomNearestNeighbor (generic function with 1 method)

## Local Search with Adjacent Cities Swap

In [23]:
function calcIndex(index, n)
    result = 0
    if index <= 0
        result = n-(abs(index)%n)
    else
        result = (((index)-1)%(n))+1
    end
    result
end

function localSearchAdjacentCitiesSwap(I::Instance, s₀::Solution)
    s = copy(s₀)
    i=1
    v=0.0
    while i<I.n
        i=2
        while i<I.n
            
            
#             v=s.v-I.D[s.π[a],s.π[a+1]]-I.D[s.π[b],s.π[b+1]]+I.D[s.π[a],s.π[b]]+I.D[s.π[a+1],s.π[b+1]]            
#             v=s.v-I.D[s.π[i-1],s.π[i]]-I.D[s.π[i+1],s.π[i+2]]+I.D[s.π[i-1],s.π[i+1]]+I.D[s.π[i],s.π[i+2]]

            a = i-1
            b = i+1
            println("A: $(a) B: $(b)")
            
            newπ = copy(s.π) 
            newπ[i],newπ[i+1] = newπ[i+1],newπ[i]

#             v = 0
#             for lookahead=1:I.l
#                 for i=1:I.n
#                     v += I.D[newπ[i],newπ[((i+lookahead-1)%(I.n))+1]]
#                 end
#             end
            
            v = s.v
            # diminuir a solução antiga
            # acrescentar a nova solução
            
            println("Pi anterior: $(s.π)")
            println("Pi novo: $(newπ)")
            
            for l1=1:I.l
                for l2=1:I.l
                    indexA1 = calcIndex(a-l1-1, I.n)
                    indexA2 = calcIndex(a-l1-1+l2, I.n)
                    indexB1 = calcIndex(b-l1-1, I.n)
                    indexB2 = calcIndex(b-l1-1+l2, I.n)
                    
                    println("Removendo $(indexA1)-$(indexA2): $(I.D[s.π[indexA1],s.π[indexA2]])")    
                    v -= I.D[s.π[indexA1],s.π[indexA2]] 
                    println("Removendo $(indexB1)-$(indexB2): $(I.D[s.π[indexB1],s.π[indexB2]])")  
                    v -= I.D[s.π[indexB1],s.π[indexB2]]
                    
                    println("Adicionando $(indexA1)-$(indexA2): $(I.D[newπ[indexA1],newπ[indexA2]])")  
                    v += I.D[newπ[indexA1],newπ[indexA2]]
                    println("Adicionando $(indexB1)-$(indexB2): $(I.D[newπ[indexB1],newπ[indexB2]])")  
                    v += I.D[newπ[indexB1],newπ[indexB2]]
                end
            end
            
            println("Comparar valores: $(s.v) -> $(v) ?\n\n\n\n\n\n")
            v<s.v && break ## first improvement
            i += 1
        end
        if i<I.n
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i]
            s.v = v
        end
    end
    s
end

localSearchAdjacentCitiesSwap (generic function with 1 method)

## Local Search with 2-exchange

In [50]:
function localSearch2Exchange(I::Instance,s₀::Solution)
    s=copy(s₀)
    move=(1,1)
    v=0.0
    while move!=(0,0)
        move=(0,0)
        for a=1:I.n, b=(a+2):I.n # reduzir para vizinhos mais próximos (a -> bs 10-20 para frente) (n^2 para 10^n) |a-b| <= 10
            # |a-b| <= 10 else return
#             I.D[s.π[a],s.π[b]] > I.avg/2 && continue
            a==1 && b==I.n && continue
            
            newπ = copy(s.π) 
            aux = newπ[a+1] #swap
            newπ[a+1] = newπ[b]
            newπ[b] = aux
            
#             println(s.π)
#             println(newπ)

#             v = 0
#             for lookahead=1:I.l
#                 for i=1:I.n
#                     v += I.D[newπ[i],newπ[((i+lookahead-1)%(I.n))+1]]
#                 end
#             end
            v = s.v
            # diminuir a solução antiga
            # acrescentar a nova solução
            for l1=1:I.l
                for l2=1:I.l
                    indexA1 = calcIndex(a-l1-1, I.n)
                    indexA2 = calcIndex(a-l1-1+l2, I.n)
                    indexB1 = calcIndex(b-l1-1, I.n)
                    indexB2 = calcIndex(b-l1-1+l2, I.n)
                        
                    v -= I.D[s.π[indexA1],s.π[indexA2]] 
                    v -= I.D[s.π[indexB1],s.π[indexB2]]
                    
                    v += I.D[newπ[indexA1],newπ[indexA2]]
                    v += I.D[newπ[indexB1],newπ[indexB2]]
                end
            end
        
            
#             v=s.v-I.D[s.π[a],s.π[a+1]]-I.D[s.π[b],s.π[b+1]]+I.D[s.π[a],s.π[b]]+I.D[s.π[a+1],s.π[b+1]]
        
            if v<s.v
                move=(a,b)
#                 println("\n\n\n1")
#                 println(newπ)
                break ## first improvement
            end
        end
        if move!=(0,0)
            ## reverse segment
            move=(move[1]+1,move[2])
            while move[1]<move[2]
                s.π[move[1]],s.π[move[2]] = s.π[move[2]],s.π[move[1]]
                move=(move[1]+1,move[2]-1)
            end
#             println("2")
#             println(s.π)
            s.v=v
        end
    end
    s
end

localSearch2Exchange (generic function with 1 method)

In [54]:
I = Instance("instances/test.txt")
b = nearestNeighbor(I)
println("Nearest Neighbor: $(b.v).")

nIterations = 5 # DEFINIR ITERAÇÕES!
k = 10 # DEFINIR K!

println("Local Search Adjacent Cities Swap")
for i = 1:nIterations
    s = randomNearestNeighbor(I,k) 
    s = localSearchAdjacentCitiesSwap(I,s)
    if s.v < b.v
        b = copy(s)
    end
    println("Iteration $i: $(b.v)")
end
println("Best solution $(b.v).")

# println("Local Search with 2-Exchange")
# b = nearestNeighbor(I)
# for i = 1:nIterations
#     println("Iteration $i: $(b.v)")
#     s = randomNearestNeighbor(I,k) 
#     s = localSearch2Exchange(I,s)
#     if s.v < b.v
#         b = copy(s)
#     end
# end
# println("Best solution $(b.v).")


Nearest Neighbor: 8.0.
Local Search Adjacent Cities Swap
A: 1 B: 3
Pi anterior: [1, 3, 5, 4, 2]
Pi novo: [1, 5, 3, 4, 2]
Removendo 4-5: 1.0
Removendo 1-2: 2.0
Adicionando 4-5: 1.0
Adicionando 1-2: 1.0
Comparar valores: 9.0 -> 8.0 ?






A: 1 B: 3
Pi anterior: [1, 5, 3, 4, 2]
Pi novo: [1, 3, 5, 4, 2]
Removendo 4-5: 1.0
Removendo 1-2: 1.0
Adicionando 4-5: 1.0
Adicionando 1-2: 2.0
Comparar valores: 8.0 -> 9.0 ?






A: 2 B: 4
Pi anterior: [1, 5, 3, 4, 2]
Pi novo: [1, 5, 4, 3, 2]
Removendo 5-1: 2.0
Removendo 2-3: 3.0
Adicionando 5-1: 2.0
Adicionando 2-3: 1.0
Comparar valores: 8.0 -> 6.0 ?






A: 1 B: 3
Pi anterior: [1, 5, 4, 3, 2]
Pi novo: [1, 4, 5, 3, 2]
Removendo 4-5: 1.0
Removendo 1-2: 1.0
Adicionando 4-5: 1.0
Adicionando 1-2: 3.0
Comparar valores: 6.0 -> 8.0 ?






A: 2 B: 4
Pi anterior: [1, 5, 4, 3, 2]
Pi novo: [1, 5, 3, 4, 2]
Removendo 5-1: 2.0
Removendo 2-3: 1.0
Adicionando 5-1: 2.0
Adicionando 2-3: 3.0
Comparar valores: 6.0 -> 8.0 ?






A: 3 B: 5
Pi anterior: [1, 5, 4, 3, 2]

In [49]:
println(b.π)

[1, 5, 4, 2, 3]


In [ ]:
using Random
Random.seed!(1234)